In [1]:
import pandas as pd
import requests as req
from bs4 import BeautifulSoup
import time
import numpy as np
import lxml
import chardet

#!pip install html5lib
#import html5lib



In [2]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}


In [3]:
years = list(range(2022, 2017, -1))

linkApertura ="https://transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA/saison_id/"
linkClausura ="https://transfermarkt.com/liga-mx-clausura/startseite/wettbewerb/MEX1/saison_id/"
links = [linkApertura , linkClausura]


In [4]:
def getSeasonLeagueTeamsUrls(ligaMxUrl):
    data = req.get(ligaMxUrl, headers = headers)    
    soup = BeautifulSoup(data.text, "lxml")
    teams_table=soup.select('table.items')[0]

    #Retrieving team links
    links = teams_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/startseite/verein/' in l]

    league_urls = [f"https://transfermarkt.com{l}" for l in links]
    return league_urls, soup


In [5]:
def extract_df_from_tMarkt_table(urlList, years):
    
    def extractTournamentNameFromLink(url):
        year = url[-4: ]
        tournamentName = "Apertura" if 'MEXA' in url else "Clausura"
        years = year + "/" + str( int(year) + 1)
        return tournamentName , years
        

    teamsDescriptionList = []
    
    for year in years:
        for page in urlList:
            page = page + str(year)
            data = req.get(page, headers = headers)    
            
            team_features = pd.read_html(data.text ,match = 'Squad')[0]
            
            #These lines fix the mismatch between headers and columns when 
            #exportring to dataframe
            del team_features["Club"]
            del team_features["Total market value"]
    
            team_features = team_features.drop(team_features.index[-1])

            #New column names            
            team_features.columns = ["club","squad","avgAge","foreigners","avg_market_value","total_market_value"]
            
            #Adding some extra columns
            team_features["year"]= year
            
            tournamentName, yearsPeriod = extractTournamentNameFromLink(page)
            team_features["tournament"]= tournamentName
            team_features["period"]= yearsPeriod
            
            #Adding dataframe from this page to list
            teamsDescriptionList.append(team_features)
            time.sleep(1)        
    
    
    teamsInfo = pd.concat(teamsDescriptionList)
    
    #Erasing Euro symbol 
    teamsInfo['avg_market_value']   = teamsInfo['avg_market_value'  ].str.replace('\u20ac','')
    teamsInfo['total_market_value'] = teamsInfo['total_market_value'].str.replace('\u20ac','')
    
     
    return teamsInfo
    

In [6]:
teamsInfo = extract_df_from_tMarkt_table(links, years)

In [7]:
teamsInfo[teamsInfo["club"] == "Deportivo Guadalajara"]

,club,squad,avgAge,foreigners,avg_market_value,total_market_value,year,tournament,period
3,Deportivo Guadalajara,26.0,26.1,0,2.52m,65.60m,2022,Apertura,2022/2023
3,Deportivo Guadalajara,26.0,26.1,0,2.52m,65.60m,2022,Clausura,2022/2023
4,Deportivo Guadalajara,53.0,24.1,1,1.49m,78.85m,2021,Apertura,2021/2022
4,Deportivo Guadalajara,53.0,24.1,1,1.49m,78.85m,2021,Clausura,2021/2022
5,Deportivo Guadalajara,39.0,25.5,0,1.64m,64.10m,2020,Apertura,2020/2021
5,Deportivo Guadalajara,39.0,25.5,0,1.64m,64.10m,2020,Clausura,2020/2021
4,Deportivo Guadalajara,42.0,24.8,1,1.55m,65.13m,2019,Apertura,2019/2020
4,Deportivo Guadalajara,42.0,24.8,1,1.55m,65.13m,2019,Clausura,2019/2020
5,Deportivo Guadalajara,56.0,24.3,1,1.09m,61.30m,2018,Apertura,2018/2019
5,Deportivo Guadalajara,56.0,24.3,1,1.09m,61.30m,2018,Clausura,2018/2019


In [8]:
teamsInfo.to_csv("teamsFeatures.csv", encoding ="latin-1")

0     3.51m
1     3.21m
2     2.93m
3     2.52m
4     2.25m
      ...  
13     992k
14     618k
15     835k
16     540k
17     634k
Name: avg_market_value, Length: 90, dtype: object

In [47]:
#teams_table=soup.select('table.items')[0]
#team_features = pd.read_html(str(teams_table))

league_urls, soup_clausura = getSeasonLeagueTeamsUrls("https://transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA/saison_id/2022")
league_urls =list(set(league_urls))
    

In [72]:
headlines = soup_clausura.select("h2.content-box-headline")
#headlines = headlines.select("h2.content-box-headline")

#headlines = [l.get("class") for l in headlines]
headlines

#headlines = [l.get("class") for l in headlines]
#headlines = [l for l in headlines if l and 'all_comps/shooting/' in l]

#element = soup_clausura.select('h2.content-box-headline')
#element
#soup_clausura

[<h2 class="content-box-headline">Info</h2>,
 <h2 class="content-box-headline">
         Clubs - Liga MX Apertura 22/23    </h2>,
 <h2 class="content-box-headline">Top goalscorers</h2>,
 <h2 class="content-box-headline">More News</h2>,
 <h2 class="content-box-headline">Most valuable players in the league</h2>]

In [75]:
type(headlines)

bs4.element.ResultSet

In [136]:
#headline = [h for h in headlines if re.compile('^*content*') in h]

for h in list(headlines):
    #if re.match('^More News*', h):
    if 'Club' in h.string:
        temp = h.string.strip()
        temp = temp.replace('Clubs - Liga MX',"")
        print(temp.strip())
#headline

Apertura 22/23


In [126]:
headlines1 = soup_clausura.find_all('h2', href="content-box-headline")
headlines1

[]

In [ ]:
teams_table=soup_clausura.select('table.items')[0]

In [ ]:
test_data = req.get(pageClausura, headers = headers) #This was used 

In [ ]:
team_features = pd.read_html(test_data.text ,match = 'Squad')[0]


In [ ]:
del team_features["Club"]
del team_features["Total market value"]
team_features = team_features.drop(team_features.index[-1])


In [ ]:
team_features.columns = ["Club","squad","avgAge","foreigners","avg_market_value","total_market_value"]

In [22]:
team_features

NameError: name 'team_features' is not defined

In [ ]:
#teams_table
league_urls =list(set(league_urls))


In [ ]:
#teams_table
#soup_clausura

#team_features = pd.read_html(str(teams_table))[0]
team_features = pd.read_html(soup_clausura,match = 'Squad')[0]

#match = "%Clubs%Liga MX Clausura 22/23%"

In [ ]:
team_features

In [ ]:
teams_table

In [ ]:
from urllib import request as u_r

def strip_website():

    request = u_r.Request("https://www.transfermarkt.com/marco-reus/verletzungen/spieler/35207")
    request.add_header('User-Agent', 'my-cool-app')
    with u_r.urlopen(request) as f:
        pass

In [ ]:
strip_website

In [ ]:
with u_r.urlopen("https://www.transfermarkt.com") as response:
    html = response.read

In [ ]:
'''
#Retrieving team links
links = teams_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/startseite/verein/' in l]

league_urls = [f"https://transfermarkt.com{l}" for l in links]
'''

In [156]:
url = "https://transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEXA/saison_id/2022"
year = url[-4: ]
#type(year)
year + "/" + str( int(year) + 1)

'2022/2023'

In [159]:
def extractTournamentName(url):
    year = url[-4: ]
    tournamentName = "Apertura " if 'MEXA' in url else "Clausura "
    years = year + "/" + str( int(year) + 1)
    return tournamentName + years

In [161]:
extractTournamentName("https://transfermarkt.com/liga-mx-apertura/startseite/wettbewerb/MEX1/saison_id/2022")

'Clausura 2022/2023'